In [41]:
import requests
import json
from time import sleep
import time
from bs4 import BeautifulSoup
from pathlib import Path
import numpy as np
import datetime
import os
import re
import glob
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import pandas as pd
import re
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Normalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import random

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

now = str(datetime.datetime.now().year)+str('0'+str(datetime.datetime.now().month))[-2:]+str('0'+str(datetime.datetime.now().day))[-2:]


In [42]:
# Deep neural network regression

In [43]:
df = pd.read_csv('datasets/20250623_dataset_cleaned.csv')

In [44]:
df.columns

Index(['make', 'year', 'trim', 'model', 'stock_type', 'fuel_type', 'bodystyle',
       'photo_count', 'drivetrain', 'engine', 'accidents_or_damage',
       'clean_title', 'one_owner_vehicle', 'personal_use_only', 'open_recall',
       'comfort_rating', 'interior_rating', 'performance_rating',
       'value_rating', 'exterior_rating', 'reliability_rating',
       'interior_color_tone', 'exterior_color_tone', 'transmission_type',
       'mileage_clean', 'price'],
      dtype='object')

In [45]:
df.describe(include='all')

,make,year,trim,model,stock_type,fuel_type,bodystyle,photo_count,drivetrain,engine,accidents_or_damage,clean_title,one_owner_vehicle,personal_use_only,open_recall,comfort_rating,interior_rating,performance_rating,value_rating,exterior_rating,reliability_rating,interior_color_tone,exterior_color_tone,transmission_type,mileage_clean,price
count,30232,30232.000000,30232,30232,30232,30232,30232,30232.000000,30232,30232,30232,30232,30232,30232,30232,30232.000000,30232.000000,30232.000000,30232.000000,30232.000000,30232.000000,30232,30232,30232,30232.000000,30232.000000
unique,8,NaN,610,194,2,3,11,NaN,5,1199,3,3,3,3,2,NaN,NaN,NaN,NaN,NaN,NaN,7,13,7,NaN,NaN
top,nissan,NaN,sv,cr_v,used,Gasoline,suv,NaN,FWD,"2.5L I-4 DOHC, variable valve control, regular...",At least 1 accident or damage reported,Unknown,No,Yes,None,NaN,NaN,NaN,NaN,NaN,NaN,Black,Silver,Automatic,NaN,NaN
freq,8593,NaN,2647,1882,30231,29418,13837,NaN,15603,1786,15041,20344,19642,21465,25776,NaN,NaN,NaN,NaN,NaN,NaN,17760,8508,25074,NaN,NaN
mean,NaN,2015.897989,NaN,NaN,NaN,NaN,NaN,25.715798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.595303,4.531771,4.517581,4.548991,4.636432,4.664336,NaN,NaN,NaN,106362.869104,13672.385155
std,NaN,4.572735,NaN,NaN,NaN,NaN,NaN,13.518752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.234643,0.230748,0.276783,0.246834,0.221085,0.234620,NaN,NaN,NaN,49953.639064,4525.612866
min,NaN,1976.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,2013.000000,NaN,NaN,NaN,NaN,NaN,18.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,4.400000,4.400000,4.500000,4.500000,4.600000,NaN,NaN,NaN,71496.250000,9998.000000
50%,NaN,2016.000000,NaN,NaN,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,4.600000,4.600000,4.600000,4.700000,4.700000,NaN,NaN,NaN,101579.000000,14290.000000
75%,NaN,2019.000000,NaN,NaN,NaN,NaN,NaN,33.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.800000,4.700000,4.700000,4.700000,4.800000,4.800000,NaN,NaN,NaN,134777.000000,17507.000000


In [46]:
# Define vocab/embedding sizes
num_makes = len(df['make'].unique())
num_stock_types = len(df['stock_type'].unique())
num_fuel_types = len(df['fuel_type'].unique())
num_bodystyles = len(df['bodystyle'].unique())
num_drivetrains = len(df['drivetrain'].unique())
num_accidents_or_damages = len(df['accidents_or_damage'].unique())
num_clean_titles = len(df['clean_title'].unique())
num_one_owner_vehicles = len(df['one_owner_vehicle'].unique())
num_personal_use_onlys = len(df['personal_use_only'].unique())
num_open_recalls = len(df['open_recall'].unique())
num_interior_color_tones = len(df['interior_color_tone'].unique())
num_exterior_color_tones = len(df['exterior_color_tone'].unique())
num_transmission_types = len(df['transmission_type'].unique())

num_trim_tokens = 1000
num_model_tokens = 1000
num_engine_tokens = 1000

embedding_dim = 8

# Input layers
############### Categorical
# make
# stock_type
# fuel_type
# bodystyle
# drivetrain
# accidents_or_damage
# clean_title
# one_owner_vehicle
# personal_use_only
# open_recall
# interior_color_tone
# exterior_color_tone
# transmission_type
make_input = Input(shape=(1,), name='make_input')      # integer encoded
stock_type_input = Input(shape=(1,), name='stock_type_input')      # integer encoded
fuel_type_input = Input(shape=(1,), name='fuel_type_input')      # integer encoded
bodystyle_input = Input(shape=(1,), name='bodystyle_input')      # integer encoded
drivetrain_input = Input(shape=(1,), name='drivetrain_input')      # integer encoded
accidents_or_damage_input = Input(shape=(1,), name='accidents_or_damage_input')      # integer encoded
clean_title_input = Input(shape=(1,), name='clean_title_input')      # integer encoded
one_owner_vehicle_input = Input(shape=(1,), name='one_owner_vehicle_input')      # integer encoded
personal_use_only_input = Input(shape=(1,), name='personal_use_only_input')      # integer encoded
open_recall_input = Input(shape=(1,), name='open_recall_input')      # integer encoded
interior_color_tone_input = Input(shape=(1,), name='interior_color_tone_input')      # integer encoded
exterior_color_tone_input = Input(shape=(1,), name='exterior_color_tone_input')      # integer encoded
transmission_type_input = Input(shape=(1,), name='transmission_type_input')      # integer encoded

############### Word embedding
# trim
# model
# engine
trim_input = Input(shape=(5,), name='trim_input') # tokenized sequence
model_input = Input(shape=(5,), name='model_input') # tokenized sequence
engine_input = Input(shape=(5,), name='engine_input') # tokenized sequence

############### Continuous
# year
# photo_count
# comfort_rating
# interior_rating
# performance_rating
# value_rating
# exterior_rating
# reliability_rating
# mileage_clean
year_input = Input(shape=(1,), name='year_input')
photo_count_input = Input(shape=(1,), name='photo_count_input')
comfort_rating_input = Input(shape=(1,), name='comfort_rating_input')
interior_rating_input = Input(shape=(1,), name='interior_rating_input')
performance_rating_input = Input(shape=(1,), name='performance_rating_input')
value_rating_input = Input(shape=(1,), name='value_rating_input')
exterior_rating_input = Input(shape=(1,), name='exterior_rating_input')
reliability_rating_input = Input(shape=(1,), name='reliability_rating_input')
mileage_input = Input(shape=(1,), name='mileage_input')

# Embedding layers
make_emb = Flatten()(Embedding(num_makes, embedding_dim)(make_input))
stock_type_emb = Flatten()(Embedding(num_stock_types, embedding_dim)(stock_type_input))
fuel_type_emb = Flatten()(Embedding(num_fuel_types, embedding_dim)(fuel_type_input))
bodystyle_emb = Flatten()(Embedding(num_bodystyles, embedding_dim)(bodystyle_input))
drivetrain_emb = Flatten()(Embedding(num_drivetrains, embedding_dim)(drivetrain_input))
accidents_or_damage_emb = Flatten()(Embedding(num_accidents_or_damages, embedding_dim)(accidents_or_damage_input))
clean_title_emb = Flatten()(Embedding(num_clean_titles, embedding_dim)(clean_title_input))
one_owner_vehicle_emb = Flatten()(Embedding(num_one_owner_vehicles, embedding_dim)(one_owner_vehicle_input))
personal_use_only_emb = Flatten()(Embedding(num_personal_use_onlys, embedding_dim)(personal_use_only_input))
open_recall_emb = Flatten()(Embedding(num_open_recalls, embedding_dim)(open_recall_input))
interior_color_tone_emb = Flatten()(Embedding(num_interior_color_tones, embedding_dim)(interior_color_tone_input))
exterior_color_tone_emb = Flatten()(Embedding(num_exterior_color_tones, embedding_dim)(exterior_color_tone_input))
transmission_type_emb = Flatten()(Embedding(num_transmission_types, embedding_dim)(transmission_type_input))
trim_emb = Flatten()(Embedding(num_trim_tokens, embedding_dim)(trim_input))
model_emb = Flatten()(Embedding(num_model_tokens, embedding_dim)(model_input))
engine_emb = Flatten()(Embedding(num_engine_tokens, embedding_dim)(engine_input))

# Normalize continuous variables
norm_year_layer = Normalization()
norm_year = norm_year_layer(year_input)
norm_photo_count_layer = Normalization()
norm_photo_count = norm_photo_count_layer(photo_count_input)
norm_comfort_rating_layer = Normalization()
norm_comfort_rating = norm_comfort_rating_layer(comfort_rating_input)
norm_interior_rating_layer = Normalization()
norm_interior_rating = norm_interior_rating_layer(interior_rating_input)
norm_performance_rating_layer = Normalization()
norm_performance_rating = norm_performance_rating_layer(performance_rating_input)
norm_value_rating_layer = Normalization()
norm_value_rating = norm_value_rating_layer(value_rating_input)
norm_exterior_rating_layer = Normalization()
norm_exterior_rating = norm_exterior_rating_layer(exterior_rating_input)
norm_reliability_rating_layer = Normalization()
norm_reliability_rating = norm_reliability_rating_layer(reliability_rating_input)
norm_mileage_layer = Normalization()
norm_mileage = norm_mileage_layer(mileage_input)

# Concatenate features
x = Concatenate()([make_emb,stock_type_emb,fuel_type_emb,bodystyle_emb,
                   drivetrain_emb,accidents_or_damage_emb,clean_title_emb,
                   one_owner_vehicle_emb,personal_use_only_emb,open_recall_emb,
                   interior_color_tone_emb,exterior_color_tone_emb,transmission_type_emb,
                   trim_emb,model_emb,engine_emb,norm_year,norm_photo_count,
                   norm_comfort_rating,norm_interior_rating,norm_performance_rating,
                   norm_value_rating,norm_exterior_rating,norm_reliability_rating,norm_mileage])

# Dense layers
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)

# Build and compile model
model = Model(inputs=[
    make_input,stock_type_input,fuel_type_input,bodystyle_input,drivetrain_input,
    accidents_or_damage_input,clean_title_input,one_owner_vehicle_input,
    personal_use_only_input,open_recall_input,interior_color_tone_input,
    exterior_color_tone_input,transmission_type_input,trim_input,model_input,
    engine_input,year_input,photo_count_input,comfort_rating_input,interior_rating_input,
    performance_rating_input,value_rating_input,exterior_rating_input,reliability_rating_input,
    mileage_input,], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [47]:
make_le = LabelEncoder()
stock_type_le = LabelEncoder()
fuel_type_le = LabelEncoder()
bodystyle_le = LabelEncoder()
drivetrain_le = LabelEncoder()
accidents_or_damage_le = LabelEncoder()
clean_title_le = LabelEncoder()
one_owner_vehicle_le = LabelEncoder()
personal_use_only_le = LabelEncoder()
open_recall_le = LabelEncoder()
interior_color_tone_le = LabelEncoder()
exterior_color_tone_le = LabelEncoder()
transmission_type_le = LabelEncoder()

df['make_encoded'] = make_le.fit_transform(df['make'])
df['stock_type_encoded'] = stock_type_le.fit_transform(df['stock_type'])
df['fuel_type_encoded'] = fuel_type_le.fit_transform(df['fuel_type'])
df['bodystyle_encoded'] = bodystyle_le.fit_transform(df['bodystyle'])
df['drivetrain_encoded'] = drivetrain_le.fit_transform(df['drivetrain'])
df['accidents_or_damage_encoded'] = accidents_or_damage_le.fit_transform(df['accidents_or_damage'])
df['clean_title_encoded'] = clean_title_le.fit_transform(df['clean_title'])
df['one_owner_vehicle_encoded'] = one_owner_vehicle_le.fit_transform(df['one_owner_vehicle'])
df['personal_use_only_encoded'] = personal_use_only_le.fit_transform(df['personal_use_only'])
df['open_recall_encoded'] = open_recall_le.fit_transform(df['open_recall'])
df['interior_color_tone_encoded'] = interior_color_tone_le.fit_transform(df['interior_color_tone'])
df['exterior_color_tone_encoded'] = exterior_color_tone_le.fit_transform(df['exterior_color_tone'])
df['transmission_type_encoded'] = transmission_type_le.fit_transform(df['transmission_type'])

# Tokenize and pad 'trim' text
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['trim'])
trim_seq = tokenizer.texts_to_sequences(df['trim'])
trim_padded = pad_sequences(trim_seq, maxlen=5, padding='post')

# Tokenize and pad 'model' text
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['model'])
model_seq = tokenizer.texts_to_sequences(df['model'])
model_padded = pad_sequences(model_seq, maxlen=5, padding='post')

# Tokenize and pad 'engine' text
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['engine'])
engine_seq = tokenizer.texts_to_sequences(df['engine'])
engine_padded = pad_sequences(engine_seq, maxlen=5, padding='post')


In [48]:
X_make = df['make_encoded'].values.reshape(-1, 1)
X_stock_type = df['stock_type_encoded'].values.reshape(-1, 1)
X_fuel_type = df['fuel_type_encoded'].values.reshape(-1, 1)
X_bodystyle = df['bodystyle_encoded'].values.reshape(-1, 1)
X_drivetrain = df['drivetrain_encoded'].values.reshape(-1, 1)
X_accidents_or_damage = df['accidents_or_damage_encoded'].values.reshape(-1, 1)
X_clean_title = df['clean_title_encoded'].values.reshape(-1, 1)
X_one_owner_vehicle = df['one_owner_vehicle_encoded'].values.reshape(-1, 1)
X_personal_use_only = df['personal_use_only_encoded'].values.reshape(-1, 1)
X_open_recall = df['open_recall_encoded'].values.reshape(-1, 1)
X_interior_color_tone = df['interior_color_tone_encoded'].values.reshape(-1, 1)
X_exterior_color_tone = df['exterior_color_tone_encoded'].values.reshape(-1, 1)
X_transmission_type = df['transmission_type_encoded'].values.reshape(-1, 1)

X_trim = trim_padded  # shape (N, 5)
X_model = model_padded  # shape (N, 5)
X_engine = engine_padded  # shape (N, 5)

X_year = df['year'].values.reshape(-1, 1)
X_photo_count = df['photo_count'].values.reshape(-1, 1)
X_comfort_rating = df['comfort_rating'].values.reshape(-1, 1)
X_interior_rating = df['interior_rating'].values.reshape(-1, 1)
X_performance_rating = df['performance_rating'].values.reshape(-1, 1)
X_value_rating = df['value_rating'].values.reshape(-1, 1)
X_exterior_rating = df['exterior_rating'].values.reshape(-1, 1)
X_reliability_rating = df['reliability_rating'].values.reshape(-1, 1)
X_mileage = df['mileage_clean'].values.reshape(-1, 1)

y = df['price'].values.reshape(-1, 1)

# Train-test split
(X_make_train, X_make_val,
 X_stock_type_train, X_stock_type_val,
 X_fuel_type_train, X_fuel_type_val,
 X_bodystyle_train, X_bodystyle_val,
 X_drivetrain_train, X_drivetrain_val,
 X_accidents_or_damage_train, X_accidents_or_damage_val,
 X_clean_title_train, X_clean_title_val,
 X_one_owner_vehicle_train, X_one_owner_vehicle_val,
 X_personal_use_only_train, X_personal_use_only_val,
 X_open_recall_train, X_open_recall_val,
 X_interior_color_tone_train, X_interior_color_tone_val,
 X_exterior_color_tone_train, X_exterior_color_tone_val,
 X_transmission_type_train, X_transmission_type_val,
 X_trim_train, X_trim_val,
 X_model_train, X_model_val,
 X_engine_train, X_engine_val,
 X_year_train, X_year_val,
 X_photo_count_train, X_photo_count_val,
 X_comfort_rating_train, X_comfort_rating_val,
 X_interior_rating_train, X_interior_rating_val,
 X_performance_rating_train, X_performance_rating_val,
 X_value_rating_train, X_value_rating_val,
 X_exterior_rating_train, X_exterior_rating_val,
 X_reliability_rating_train, X_reliability_rating_val,
 X_mileage_train, X_mileage_val,
 y_train, y_val) = train_test_split(
    X_make,X_stock_type,X_fuel_type,X_bodystyle,
    X_drivetrain,X_accidents_or_damage,X_clean_title,X_one_owner_vehicle,
    X_personal_use_only,X_open_recall,X_interior_color_tone,X_exterior_color_tone,
    X_transmission_type,X_trim,X_model,X_engine,X_year,X_photo_count,X_comfort_rating,
    X_interior_rating,X_performance_rating,X_value_rating,X_exterior_rating,
    X_reliability_rating,X_mileage, y, test_size=0.2, random_state=2)

In [ ]:
history = model.fit(
    x=[X_make_train,X_stock_type_train,X_fuel_type_train,X_bodystyle_train,
       X_drivetrain_train,X_accidents_or_damage_train,X_clean_title_train,
       X_one_owner_vehicle_train,X_personal_use_only_train,X_open_recall_train,
       X_interior_color_tone_train,X_exterior_color_tone_train,X_transmission_type_train,
       X_trim_train,X_model_train,X_engine_train,
       X_year_train,X_photo_count_train,
       X_comfort_rating_train,X_interior_rating_train,X_performance_rating_train,
       X_value_rating_train,X_exterior_rating_train,X_reliability_rating_train,X_mileage_train],
    y=y_train,
    validation_data=([X_make_val,X_stock_type_val,X_fuel_type_val,X_bodystyle_val,
                      X_drivetrain_val,X_accidents_or_damage_val,X_clean_title_val,
                      X_one_owner_vehicle_val,X_personal_use_only_val,X_open_recall_val,
                      X_interior_color_tone_val,X_exterior_color_tone_val,X_transmission_type_val,
                      X_trim_val,X_model_val,X_engine_val,
                      X_year_val,X_photo_count_val,
                      X_comfort_rating_val,X_interior_rating_val,X_performance_rating_val,
                      X_value_rating_val,X_exterior_rating_val,X_reliability_rating_val,X_mileage_val], y_val),
    epochs=30,
    batch_size=32,
)


Epoch 1/30
756/756 [==============================] - 4s 3ms/step - loss: 54140892.0000 - mae: 5478.0059 - val_loss: 14285362.0000 - val_mae: 2886.7322
Epoch 2/30
756/756 [==============================] - 2s 3ms/step - loss: 11763967.0000 - mae: 2727.1118 - val_loss: 12962464.0000 - val_mae: 2734.6072
Epoch 3/30
756/756 [==============================] - 2s 2ms/step - loss: 10002127.0000 - mae: 2501.9832 - val_loss: 12266864.0000 - val_mae: 2659.5154
Epoch 4/30
756/756 [==============================] - 2s 3ms/step - loss: 8606919.0000 - mae: 2299.0762 - val_loss: 8799183.0000 - val_mae: 2152.9309
Epoch 5/30
756/756 [==============================] - 2s 3ms/step - loss: 7632499.0000 - mae: 2151.1025 - val_loss: 9884353.0000 - val_mae: 2314.2371
Epoch 6/30
199/756 [======>.......................] - ETA: 1s - loss: 7249841.5000 - mae: 2087.3955

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()

# Plot training & validation MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('MAE Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
predictions = model.predict(df_test)